In [ ]:
# BACKTEST VERSION 1.0 ========================================================
# Apply market indicators and basic commonality analysis
# to detect bottom and trigger the buying time via Telegram
# also perform back-testing record -> Save to Picture
# =============================================================================
# Library for process Datetime
from datetime import datetime, timedelta
import time
import pytz
# Library for POST GET requests
import requests
import urllib.parse
import json
# Library for Data Analysis
import pandas as pd
import statistics
import matplotlib.pyplot as plt
# import matplotlib.dates as mdates
import os

# Define the GMT+7 timezone
timezone = pytz.timezone('Asia/Bangkok')
# Store the top 5 failure timestamp
last_failure = []
# Set the interval for the status message (in seconds)
interval = 60*60  # 1 hours in seconds
# Check point of important handling - in UNIX timestamp
check_point = time.time()

# Define classification threshold for backtesting
threshold = 0.8

# Define your quiet hours not to send Telegram
quiet_hours = [
    ((22,0),(23,59)),  # 10 PM to midnight
    ((0,0),(6,0)),      # Midnight to 6 AM
    ((8,0),(16,30))      # work time to 8 AM - 4:30 PM
]

# Function send to Telegram ===================================================
def send_telegram(message_string, max_retry=2):
    # Telegram bot token and chat ID
    TOKEN = "5614737400:AAHbvZrJbomt09EkpPuhadBCJl7NaGu6rlg"
    ID = "5559031253"
    # Flag to sent --> Will send if flag = 0 afterall
    flag_send = 0
    current_time = datetime.now(timezone)

    for start, end in quiet_hours:
        # print(start[0], end[0])
        date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
        quiet_start = current_time.replace(hour=start[0],
                                           minute=start[1],
                                           second=0,
                                           microsecond=0)
        quiet_end = current_time.replace(hour=end[0],
                                         minute=end[1],
                                         second=59,
                                         microsecond=0)
        if quiet_start <= current_time <= quiet_end:
            flag_send += 1

    current_retry = 0
    while (current_retry <= max_retry) and (flag_send == 0):
        if current_retry > 0:
            print(f"Send Telegram: Retry #{current_retry}....................")
        try:
            # URL encode the message string
            encoded_message = urllib.parse.quote(message_string)
            # Construct the URL
            url1 = f"https://api.telegram.org/bot{TOKEN}/sendMessage?"
            url2 = f"chat_id={ID}&text={encoded_message}"
            # Send the request
            response = requests.get(url1 + url2, timeout=3)
            # Raise an HTTPError for bad responses (4xx and 5xx)
            response.raise_for_status()
            return True
        except requests.exceptions.RequestException as e:
            date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
            print(date_time, f"ERROR Telegram: {e}")
            current_retry += 1
            if current_retry > max_retry:
                print(date_time, "SEND TELEGRAM Max retries reached.")
                return False

# Function avoid Telegram spam if repeated trigger ============================
def check_trigger(message_string):
    global last_failure  # Declare last_failure as global

    current_time = time.time()
    # Remove timestamps older than 300 seconds
    last_failure = [timestamp for timestamp in last_failure
                    if current_time - timestamp <= 300]
    # Add the current timestamp
    last_failure.append(current_time)
    # Check if there are more than 5 errors within 300 seconds
    if len(last_failure) > 5:
        send_telegram(message_string)
        # Reset the list after sending the message
        last_failure = []

# Function get price 1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 12h, 1d ============
def get_data(coin_name, time_unit, time_range, max_retry=7):
    url = "https://api-pro.goonus.io/perpetual/v1/klines"
    # Get the current Unix timestamp value in ms
    current_time = int(time.time()) * 1000
    # Define the parameters for the API request
    params = {
        "symbol": coin_name,
        "interval": time_unit,
        "endTime": current_time,
        # "limit": "600"
    }
    current_retry = 0
    while current_retry <= max_retry:
        if current_retry > 0:
            print(f"GET Coin {coin_name}: Retry #{current_retry}........................")
        try:
            # Try request & get the Response during 5 seconds
            response = requests.get(url, params=params, timeout=5)
            # Raise an HTTPError for bad responses (4xx and 5xx)
            response.raise_for_status()

            historical_data = response.json()
            # Map the time_range matching with the total record
            while len(historical_data) < time_range:
                time_range -= 10
            # Check if the data has sufficient 100 records
            if len(historical_data) >= 100:
                # Take the latest "time_range" elements in the list
                historical_data = historical_data[-time_range:]
                # [Time,Open,High,Low,Close,Base qty,Quote qty,Time]
                # Filter to only include 'timestamp' and 'close'
                filtered_data = [[
                    # Convert Unix timestamp -> GMT +7 hours (+25200)
                    datetime.fromtimestamp(int(item[0])/1000, tz=pytz.utc)
                    .astimezone(timezone).strftime('%Y-%m-%d %H:%M:%S'),
                    # Close price
                    float(item[4]),
                    # Percentage difference
                    round(100*(float(item[2])-float(item[3]))/float(item[1]),2),
                    float(item[2]), float(item[3])
                ] for item in historical_data]
                # [timestamp, close, %price_diff, Highest, Lowest]
                # print(filtered_data)
                return filtered_data
            else:
                date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
                message = f"Not enough data for {coin_name}"
                print(date_time, message)
                check_trigger(message)
                return None
        except requests.exceptions.RequestException as e:
            date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
            print(date_time, f"ERROR Request {coin_name}: (retry #{current_retry}): {e}")
            current_retry += 1
            if current_retry > max_retry:
                print(f"{date_time}: Max retries reached. Raising exception.")
                send_telegram(f"ERROR GET Coin {coin_name} (Limit Retried): {e}")
                return None

# Function call API to Read empty record in mySQL database ====================
def read_database(null_data = "yes", max_retry=5):
    # Define the API endpoint and parameters
    url = "https://petiteo.com/api/read_backtest.php"
    params = {
        "null_data": null_data,
    }
    current_retry = 0
    while current_retry <= max_retry:
        if current_retry > 0:
            print(f"Retry READ {current_retry}...")
        try:
            # Make the GET request
            response = requests.get(url, params=params, timeout=3)
            # Raise an HTTPError for bad responses (4xx and 5xx)
            response.raise_for_status()
            # Parse the JSON response
            json_data = response.json()
            return [
                [
                    record["coin_name"],
                    record["date_trigger"],
                    float(record["trigger_value"])
                ] for record in json_data["data"]
            ]
        except requests.RequestException as e:
            date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
            print(f"{date_time}: READ DBfailed\n{e}")
            send_telegram(f"READ DB failed #{current_retry}\n{e}")
            current_retry += 1
            if current_retry > max_retry:
                print(f"{date_time}: READ DB Max retries reached.")
                return None

# Function call API to Update in mySQL database ===============================
def update_database(coin_name, date_trigger,
                    avg_next15, min_next15, max_next15,
                    avg_next30, min_next30, max_next30,
                    avg_next45, min_next45, max_next45,
                    avg15_per, min15_per, max15_per,
                    avg30_per, min30_per, max30_per,
                    avg45_per, min45_per, max45_per,
                    result_15, result_30, result_45, max_retry=5):
    # Define the API endpoint and parameters
    url = "https://petiteo.com/api/update_backtest.php"
    params = {
        "coin_name": coin_name,
        "date_trigger": date_trigger,

        "avg_next15": avg_next15,
        "min_next15": min_next15,
        "max_next15": max_next15,
        "avg_next30": avg_next30,
        "min_next30": min_next30,
        "max_next30": max_next30,
        "avg_next45": avg_next45,
        "min_next45": min_next45,
        "max_next45": max_next45,

        "avg15_per": avg15_per,
        "min15_per": min15_per,
        "max15_per": max15_per,
        "avg30_per": avg30_per,
        "min30_per": min30_per,
        "max30_per": max30_per,
        "avg45_per": avg45_per,
        "min45_per": min45_per,
        "max45_per": max45_per,

        "result_15": result_15,
        "result_30": result_30,
        "result_45": result_45,
    }
    current_retry = 0
    while current_retry <= max_retry:
        if current_retry > 0:
            print(f"Retry Update {current_retry}...")
        try:
            # Make the GET request
            response = requests.get(url, params=params, timeout=3)
            # Raise an HTTPError for bad responses (4xx and 5xx)
            response.raise_for_status()
            # Parse the JSON response
            response_data = response.json()
            if response_data.get("status") == "success":
                print(response_data.get("message"))
                return True
            else:
                date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
                message = f"Error UPDATE: {response_data.get('message')}"
                print(date_time, message)
                send_telegram(message)
                return False
        except requests.RequestException as e:
            date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
            print(f"{date_time}: UPDATE failed\n{e}")
            send_telegram(f"UPDATE failed: #{current_retry}\n{e}")
            current_retry += 1
            if current_retry > max_retry:
                print(f"{date_time}: UPDATE Max retries reached.")
                return False

# Funtion to upload image to host =============================================
def upload_image(file_name, max_retry=5):
    # URL of your PHP upload script
    upload_url = 'https://petiteo.com/api/upload_backtest.php'
    # Upload the file
    current_retry = 0
    while current_retry <= max_retry:
        if current_retry > 0:
            print(f"Retry UPLOAD {current_retry}...")
        try:
            with open(file_name, 'rb') as file:
                response = requests.post(upload_url, files={'file': file})
            print(response.text)
            return True
        except requests.RequestException as e:
            date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
            print(f"{date_time}: UPLOAD failed\n{e}")
            send_telegram(f"UPLOAD failed\n{e}")
            current_retry += 1
            if current_retry > max_retry:
                print(f"{date_time}: UPLOAD Max retries reached")
                return None

# Process Back testing ========================================================
def back_testing():
    date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
    print(f"{date_time}: Start Backtesting")

    pending_data = read_database(null_data='yes')
    # print("Pending Data:", pending_data)
    if pending_data and len(pending_data) > 0:
        for remain in pending_data:
            datacoin_backtest = get_data(remain[0], "1m", 600)
            # print("Last Time-range return:", datetime.strptime(datacoin_backtest[0][0], '%Y-%m-%d %H:%M:%S'))
            # Trigger datetime
            trigger_datetime = datetime.strptime(remain[1], '%Y-%m-%d %H:%M:%S')
            # Calculate the end time (15 minutes after trigger_datetime)
            end_time15 = trigger_datetime + timedelta(minutes=15)
            end_time30 = trigger_datetime + timedelta(minutes=30)
            end_time45 = trigger_datetime + timedelta(minutes=45)

            try:
                # Filter out elements that are larger than the trigger_datetime within 15 minutes
                filtered_list15 = [entry for entry in datacoin_backtest if trigger_datetime < datetime.strptime(entry[0], '%Y-%m-%d %H:%M:%S') <= end_time15]
                filtered_list30 = [entry for entry in datacoin_backtest if trigger_datetime < datetime.strptime(entry[0], '%Y-%m-%d %H:%M:%S') <= end_time30]
                filtered_list45 = [entry for entry in datacoin_backtest if trigger_datetime < datetime.strptime(entry[0], '%Y-%m-%d %H:%M:%S') <= end_time45]
                print(remain, len(filtered_list45))

                # Wait until enough dataset
                if len(filtered_list45) >= 45:
                    # ['timestamp', 'close', 'percent', 'high', 'low']
                    val15_avg = [entry15[1] for entry15 in filtered_list15]
                    val15_min = [entry15[4] for entry15 in filtered_list15]
                    val15_max = [entry15[3] for entry15 in filtered_list15]
                    val30_avg = [entry30[1] for entry30 in filtered_list30]
                    val30_min = [entry30[4] for entry30 in filtered_list30]
                    val30_max = [entry30[3] for entry30 in filtered_list30]
                    val45_avg = [entry45[1] for entry45 in filtered_list45]
                    val45_min = [entry45[4] for entry45 in filtered_list45]
                    val45_max = [entry45[3] for entry45 in filtered_list45]

                    # Calculate in 15
                    avg_value15 = statistics.mean(val15_avg) if val15_avg else None
                    avg15_per = round(100 * (avg_value15 - remain[2]) / remain[2], 2)
                    min_value15 = min(val15_min) if val15_min else None
                    min15_per = round(100 * (min_value15 - remain[2]) / remain[2], 2)
                    max_value15 = max(val15_max) if val15_max else None
                    max15_per = round(100 * (max_value15 - remain[2]) / remain[2], 2)
                    # Compare threshold and Calculate Result 15:
                    res15_avg = 2 if avg15_per >= threshold else 1 if 0 < avg15_per < threshold else -1 if -threshold < avg15_per < 0 else -2 if avg15_per <= -threshold else 0
                    res15_min = 3 if min15_per >= threshold else 2 if 0 < min15_per < threshold else -2 if -threshold < min15_per < 0 else -3 if min15_per <= -threshold else 0
                    res15_max = 4 if max15_per >= threshold else 2 if 0 < max15_per < threshold else -2 if -threshold < max15_per < 0 else -4 if max15_per <= -threshold else 0
                    result_15 = res15_avg + res15_min + res15_max

                    # Calculate in 30
                    avg_value30 = statistics.mean(val30_avg) if val30_avg else None
                    avg30_per = round(100 * (avg_value30 - remain[2]) / remain[2], 2)
                    min_value30 = min(val30_min) if val30_min else None
                    min30_per = round(100 * (min_value30 - remain[2]) / remain[2], 2)
                    max_value30 = max(val30_max) if val30_max else None
                    max30_per = round(100 * (max_value30 - remain[2]) / remain[2], 2)
                    # Compare threshold and Calculate Result 30
                    res30_avg = 2 if avg30_per >= threshold else 1 if 0 < avg30_per < threshold else -1 if -threshold < avg30_per < 0 else -2 if avg30_per <= -threshold else 0
                    res30_min = 3 if min30_per >= threshold else 2 if 0 < min30_per < threshold else -2 if -threshold < min30_per < 0 else -3 if min30_per <= -threshold else 0
                    res30_max = 4 if max30_per >= threshold else 2 if 0 < max30_per < threshold else -2 if -threshold < max30_per < 0 else -4 if max30_per <= -threshold else 0
                    result_30 = res30_avg + res30_min + res30_max

                    # Calculate in 45
                    avg_value45 = statistics.mean(val45_avg) if val45_avg else None
                    avg45_per = round(100 * (avg_value45 - remain[2]) / remain[2], 2)
                    min_value45 = min(val45_min) if val45_min else None
                    min45_per = round(100 * (min_value45 - remain[2]) / remain[2], 2)
                    max_value45 = max(val45_max) if val45_max else None
                    max45_per = round(100 * (max_value45 - remain[2]) / remain[2], 2)
                    # Compare threshold and Calculate Result 45
                    res45_avg = 2 if avg45_per >= threshold else 1 if 0 < avg45_per < threshold else -1 if -threshold < avg45_per < 0 else -2 if avg45_per <= -threshold else 0
                    res45_min = 3 if min45_per >= threshold else 2 if 0 < min45_per < threshold else -2 if -threshold < min45_per < 0 else -3 if min45_per <= -threshold else 0
                    res45_max = 4 if max45_per >= threshold else 2 if 0 < max45_per < threshold else -2 if -threshold < max45_per < 0 else -4 if max45_per <= -threshold else 0
                    result_45 = res45_avg + res45_min + res45_max

                    # Update Database
                    update_database(remain[0], remain[1],
                                    avg_value15, min_value15, max_value15,
                                    avg_value30, min_value30, max_value30,
                                    avg_value45, min_value45, max_value45,
                                    avg15_per, min15_per, max15_per,
                                    avg30_per, min30_per, max30_per,
                                    avg45_per, min45_per, max45_per,
                                    result_15, result_30, result_45)
                    # Draw graph ======================================
                    # Convert to DataFrame
                    df_graph = pd.DataFrame(datacoin_backtest, columns=['timestamp', 'close', 'percent', 'high', 'low'])
                    # Limit to the latest 250 records
                    df_graph = df_graph.tail(250)
                    # Change the "timestamp" to the datetime format
                    df_graph['timestamp'] = pd.to_datetime(df_graph['timestamp'])
                    # display(df_graph)
                    # specific_time = '2023-11-17 01:00:00'
                    # print(type(trigger_datetime), type(remain[1]))
                    # specific_time = pd.to_datetime(trigger_datetime)
                    # specific_time_float = mdates.date2num(specific_time)
                    # Plot the visualization
                    plt.figure(figsize=(10, 4))
                    plt.title(f'"{remain[0]}" at "{remain[1]}" - ({res15_avg}+{res15_min}+{res15_max})/({res30_avg}+{res30_min}+{res30_max})/({res45_avg}+{res45_min}+{res45_max})')
                    plt.plot(df_graph['timestamp'], df_graph['close'], label='Close Price', color='blue')
                    # plt.xlabel('Time')
                    # plt.ylabel('Price')
                    plt.axvline(x = trigger_datetime, color = 'red', linestyle = '--', label = 'Specific Time')
                    plt.axvline(x = end_time15, color = 'black', linestyle = ':')
                    plt.axvline(x = end_time30, color = 'black', linestyle = ':')
                    # plt.axvline(x = end_time45, color = 'black', linestyle = ':')
                    plt.legend()
                    plt.grid(True)
                    # # Save the plot as a file
                    file_name = f"{remain[1].translate(str.maketrans({':': '-', ' ': '_'}))}_{remain[0]}.png"
                    print(file_name)
                    plt.savefig(file_name, dpi=100)
                    # plt.show()
                    plt.close()
                    upload_image(file_name)
                    if os.path.exists(file_name):
                        os.remove(file_name)
            except Exception as e:
                print(f"BACKTEST ERROR backtest: {e}")
                send_telegram(f"BACKTEST ERROR backtest: {e}")
    else:
        print(" ===> ERROR:", pending_data)

# MAIN PROGRAM ================================================================
if __name__ == '__main__':
    # Send log to Telegram
    date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
    message = f"{date_time}: Beta started"
    print(message)
    send_telegram(message)

    # Infinite loop to execute
    while True:
        back_testing()

        current_time = time.time()
        if (current_time - check_point) >= interval:
            # date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
            check_point = current_time
            # Process Back Testing
            send_telegram("Beta still working")

        time.sleep(600)